## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
! pip install bayesian-optimization lightgbm catboost

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
    --------------------------------------- 1.8/102.4 MB 9.2 MB/s eta 0:00:11
   - -------------------------------------- 3.9/102.4 MB 9.8 MB/s eta 0:00:11
   -- ------------------------------------- 6.0/102.4 MB 10.3 MB/s eta 0:00:10
   --- ------------------------------------ 8.7/102.4 MB 10.5 MB/s eta 0:00:09
   ---- ----------------------------------- 11.0/102.4 MB 10.7 MB/s eta 0:00:09
   ----- ---------------------------------- 13.9/102.4 MB 11.2 MB/s eta 0:00:08
   ------ --------------------------------- 16.8/102.4 MB 11.6 MB/s eta 0:00:08
   ------- -------------------------------- 19.9/102.4 MB 12.3 MB/s eta 0:00:07
   -------- ------------------------------- 22.3/102.4 MB 12.2 MB/s eta 0:00:07
   ---------- ----------------------------- 26.0/102.4 MB 12.6 MB/s eta 0:00:07
   --

In [3]:
import sys
!{sys.executable} -m pip install bayesian-optimization

  Using cached bayesian_optimization-3.1.0-py3-none-any.whl.metadata (11 kB)
Using cached bayesian_optimization-3.1.0-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: C:\Users\tanks\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [2]:
import os
os.listdir()

['.ipynb_checkpoints', 'Bayesian_optimization_case_study.ipynb', 'data']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [3]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [4]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [5]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 6.5196543 | 1.5775203 | 4.9421339 |
| 2         | 8.1832102 | 2.3262402 | 5.8569700 |
| 3         | 7.2008305 | 1.2280504 | 5.9727801 |
| 4         | 5.9478749 | 1.6644144 | 4.2834605 |
| 5         | 9.5266546 | 2.8865105 | 6.6401440 |


Great, now let's print the best parameters and the associated maximized target.

In [6]:
print(optimizer.max['params']);optimizer.max['target']

{'a': np.float64(2.886510525103981), 'b': np.float64(6.640144076020888)}


np.float64(9.526654601124868)

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [7]:
train_df = pd.read_csv('data/flight_delays_train.csv')
test_df = pd.read_csv('data/flight_delays_test.csv')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [8]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [9]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [10]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [11]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [12]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [13]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [14]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [15]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [16]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       stratified=True,
                       nfold=3)
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 5 iterations and set init_points = 2.


In [17]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=5, init_points=2)

|   iter    |  target   | num_le... | max_depth | lambda_l2 | lambda_l1 | min_ch... | min_da... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7211608 | 2195.8095 | 27.375712 | 0.0053781 | 0.0054083 | 635.16006 | 803.63664 |
| 2         | 0.7434085 | 1190.1311 | 42.421020 | 0.0394564 | 0.0214772 | 3774.6710 | 1420.8023 |
| 3         | 0.7433697 | 2736.9213 | 33.106108 | 0.0206542 | 0.0066471 | 2112.5763 | 1444.3854 |
| 4         | 0.7427522 | 2858.4694 | 50.427204 | 0.0003707 | 0.0390902 | 3971.4210 | 1361.3435 |
| 5         | 0.7030323 | 2393.9072 | 22.850385 | 0.0101163 | 0.0294107 | 2147.2287 | 242.78484 |
| 6         | 0.7433321 | 1679.4687 | 49.746071 | 0.0101291 | 0.0090384 | 3531.4353 | 1818.4235 |
| 7         | 0.7433946 | 2883.8789 | 35.936948 | 0.0290019 | 0.0168858 | 2879.0743 | 1936.7374 |


 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [18]:
lgbBO.max

{'target': np.float64(0.7434085081497518),
 'params': {'num_leaves': np.float64(1190.1311604629607),
  'max_depth': np.float64(42.4210206554832),
  'lambda_l2': np.float64(0.0394564387769446),
  'lambda_l1': np.float64(0.021477231281508247),
  'min_child_samples': np.float64(3774.6710613453783),
  'min_data_in_leaf': np.float64(1420.802380782325)}}

Review the process at each step by using the '.res[0]' function.

In [19]:
lgbBO.res[0]

{'target': np.float64(0.721160830667969),
 'params': {'num_leaves': np.float64(2195.8095126467),
  'max_depth': np.float64(27.37571250509469),
  'lambda_l2': np.float64(0.005378147788044102),
  'lambda_l1': np.float64(0.005408398773703277),
  'min_child_samples': np.float64(635.1600696727544),
  'min_data_in_leaf': np.float64(803.63664285962)}}

In [20]:
lgbBO.res

[{'target': np.float64(0.721160830667969),
  'params': {'num_leaves': np.float64(2195.8095126467),
   'max_depth': np.float64(27.37571250509469),
   'lambda_l2': np.float64(0.005378147788044102),
   'lambda_l1': np.float64(0.005408398773703277),
   'min_child_samples': np.float64(635.1600696727544),
   'min_data_in_leaf': np.float64(803.63664285962)}},
 {'target': np.float64(0.7434085081497518),
  'params': {'num_leaves': np.float64(1190.1311604629607),
   'max_depth': np.float64(42.4210206554832),
   'lambda_l2': np.float64(0.0394564387769446),
   'lambda_l1': np.float64(0.021477231281508247),
   'min_child_samples': np.float64(3774.6710613453783),
   'min_data_in_leaf': np.float64(1420.802380782325)}},
 {'target': np.float64(0.7433697591400922),
  'params': {'num_leaves': np.float64(2736.9213417099963),
   'max_depth': np.float64(33.10610805299051),
   'lambda_l2': np.float64(0.020654290872170307),
   'lambda_l1': np.float64(0.006647190609999793),
   'min_child_samples': np.float64(2

In [39]:
best_params = lgbBO.res[1]['params']
best_params['num_leaves'] = round(best_params['num_leaves'])
best_params['max_depth'] = round(best_params['max_depth'])
best_params['lambda_l2'] = float(best_params['lambda_l2'])
best_params['lambda_l1'] = float(best_params['lambda_l1'])
best_params['min_child_samples'] = round(best_params['min_child_samples'])
best_params['min_data_in_leaf'] = round(best_params['min_data_in_leaf'])
best_params

{'num_leaves': 1190,
 'max_depth': 42,
 'lambda_l2': 0.0394564387769446,
 'lambda_l1': 0.021477231281508247,
 'min_child_samples': 3775,
 'min_data_in_leaf': 1421}

In [40]:
train_data = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
lightgbm.cv(best_params, train_data, 1000, nfold=5)

{'valid l2-mean': [0.1514673014844385,
  0.1492712314645639,
  0.1474756624813826,
  0.1459835883465858,
  0.1447208447221237,
  0.14368430438754562,
  0.14273825147083763,
  0.14203481165872095,
  0.14131263694825724,
  0.1407566387281432,
  0.14027307536902556,
  0.13981088351244875,
  0.13940518774674399,
  0.13905989318724118,
  0.13873181692308204,
  0.13843060401063412,
  0.13821316559440003,
  0.13798094128760213,
  0.1377718187261209,
  0.1375809351419043,
  0.13742102035159107,
  0.13730655812435766,
  0.13716305486388758,
  0.1370540745529002,
  0.13693824934720533,
  0.13683732004959162,
  0.13675619788507867,
  0.1366839735233893,
  0.1365990356579297,
  0.13652530718212208,
  0.1364448810939738,
  0.1363728895377551,
  0.13630394612832666,
  0.13623041986760578,
  0.13615410847218748,
  0.136086752841093,
  0.13600615668002594,
  0.13595436743651518,
  0.13591292649977826,
  0.13585111472909545,
  0.135808125583542,
  0.13575011512982368,
  0.13570593956229043,
  0.1356644

In [41]:
bst = lightgbm.train(best_params, train_data)

In [43]:
y_pred = bst.predict(X_test)
y_pred

array([0.04217453, 0.06574535, 0.01996266, ..., 0.18041187, 0.11049542,
       0.08077953])

In [45]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [55]:
y_pred = np.round(y_pred)

In [57]:
print(np.mean(y_train), np.mean(y_pred))

0.19044 0.04049


In [58]:
bst = lightgbm.LGBMClassifier(
    num_leaves=best_params['num_leaves'],
    max_depth=best_params['max_depth'],
    min_child_samples=best_params['min_child_samples'],
    reg_alpha=best_params['lambda_l1'],
    reg_lambda=best_params['lambda_l2']
)

In [59]:
bst.fit(X_train, y_train)

LGBMClassifier(max_depth=42, min_child_samples=3775, num_leaves=1190,
               reg_alpha=0.021477231281508247, reg_lambda=0.0394564387769446)

In [60]:
y_tr_pred = bst.predict(X_train)
y_pred = bst.predict(X_test)

In [64]:
np.mean(y_pred)

np.float64(0.028)